In [1]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

In [34]:
'''
ДЗ 1 Кузнецов Д. Конев Т.

#           USER1, U2, U3, ...
# PRODUCT1
# P2
# P3
# ...
# ОЦЕНКИ FROM 0 TO 5
'''

user_prod_matrix = np.array([
    [0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0],
])

user_prod_matrix = np.array([
    [5, 3, 0, 5, 4],
    [4, 0, 0, 4, 5],
    [1, 1, 0, 1, 0],
    [0, 0, 5, 0, 0],
    [2, 3, 4, 1, 0]
])

user_similarity = cosine_similarity(user_prod_matrix.T)    
print('user similarity\n', np.round(user_similarity, 3))

product_similarity = cosine_similarity(user_prod_matrix)
print('prpducts similarity\n', np.round(product_similarity, 3))

np.fill_diagonal(user_similarity, 0)
np.fill_diagonal(product_similarity, 0)

u_i, u_j = np.unravel_index(np.argmax(user_similarity), user_similarity.shape)
p_i, p_j = np.unravel_index(np.argmax(product_similarity), product_similarity.shape)

print('Ближайшие пользователи', u_i+1, u_j+1)
print('Ближайшие товары', p_i+1, p_j+1)

user similarity
 [[1.    0.744 0.184 0.989 0.921]
 [0.744 1.    0.43  0.665 0.43 ]
 [0.184 0.43  1.    0.095 0.   ]
 [0.989 0.665 0.095 1.    0.953]
 [0.921 0.43  0.    0.953 1.   ]]
prpducts similarity
 [[1.    0.918 0.867 0.    0.506]
 [0.918 1.    0.612 0.    0.29 ]
 [0.867 0.612 1.    0.    0.632]
 [0.    0.    0.    1.    0.73 ]
 [0.506 0.29  0.632 0.73  1.   ]]
Ближайшие пользователи 1 4
Ближайшие товары 1 2


In [35]:
'''
ДЗ 2 Кузнецов Д. Конев Т.

Матрица из предыдущего блока
'''

#threshhold
R = 0.6
users = [f"U{i+1}" for i in range(user_similarity.shape[0])]
curr_step = 0

print('шаг ', curr_step)
print(user_similarity)

while user_similarity.shape[0] > 1:
    curr_step += 1
    print('-'*10, 'шаг ', curr_step)
    # находим наибольшее сходство
    max_idx = np.unravel_index(np.argmax(user_similarity), user_similarity.shape)
    i, j = max_idx
    max_sim = user_similarity[i, j]
    
    
    if max_sim < R:
        break

    # создаём новый кластер
    new_cluster = f"({users[i]}+{users[j]})"

    # обновляем сходства
    new_row = np.maximum(user_similarity[i, :], user_similarity[j, :])
    new_row = np.delete(new_row, [i, j])  # убираем старые элементы

    # удаляем i и j из матрицы
    user_similarity = np.delete(user_similarity, [i, j], axis=0)
    user_similarity = np.delete(user_similarity, [i, j], axis=1)

    # добавляем новую строку/столбец
    if user_similarity.shape[0] > 0:
        user_similarity = np.vstack([user_similarity, new_row])
        new_col = np.append(new_row, 0).reshape(-1, 1)
        user_similarity = np.hstack([user_similarity, new_col])

    # обновляем список пользователей
    users = [u for k, u in enumerate(users) if k not in (i, j)] + [new_cluster]

    print("Новая матрица расстояний:")
    print(np.round(user_similarity, 3))
    print(f"Текущие кластеры: {users}")

print("Финальные кластеры:")
print(users)


шаг  0
[[0.         0.7441611  0.18421252 0.98932625 0.9210626 ]
 [0.7441611  0.         0.42994462 0.66472586 0.42994462]
 [0.18421252 0.42994462 0.         0.0952651  0.        ]
 [0.98932625 0.66472586 0.0952651  0.         0.95265102]
 [0.9210626  0.42994462 0.         0.95265102 0.        ]]
---------- шаг  1
Новая матрица расстояний:
[[0.    0.43  0.43  0.744]
 [0.43  0.    0.    0.184]
 [0.43  0.    0.    0.953]
 [0.744 0.184 0.953 0.   ]]
Текущие кластеры: ['U2', 'U3', 'U5', '(U1+U4)']
---------- шаг  2
Новая матрица расстояний:
[[0.    0.43  0.744]
 [0.43  0.    0.184]
 [0.744 0.184 0.   ]]
Текущие кластеры: ['U2', 'U3', '(U5+(U1+U4))']
---------- шаг  3
Новая матрица расстояний:
[[0.   0.43]
 [0.43 0.  ]]
Текущие кластеры: ['U3', '(U2+(U5+(U1+U4)))']
---------- шаг  4
Финальные кластеры:
['U3', '(U2+(U5+(U1+U4)))']
